In [98]:
'''
Data assimilation for deep time
Stage 1:    Prior: cGENIE only
            Proxy: petmproxy3slices format database
            PSM: bayesian proxy system model
            DA: Mingsong Li, with LMR DA Core
            
            Mingsong Li
            1/15/2020
'''
# Package
import h5py
import LMR_DA
from lib import modules_nc
from lib import modules_find_layer #.find_layer as find_layer
from lib import modules_psm_linear

from netCDF4 import Dataset
import numpy as np
import numpy.ma as ma
import numpy.matlib as mat
import scipy.stats as stats
import pandas
from sys import platform as sys_pf
import yaml
import matplotlib.pyplot as plt
if sys_pf == 'darwin':
    import matplotlib
    matplotlib.use("TkAgg")
    import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.basemap import Basemap, shiftgrid, cm

try:
    import bayspline
except ImportError as e1:
    print('Warning:', e1)
try:
    import bayspar
except ImportError as e2:
    print('Warning:', e2)
try:
    import bayfox
except ImportError as e3:
    print('Warning:', e3)
try:
    import baymag
except ImportError as e4:
    print('Warning:', e4)

In [99]:
# read DTDA-config.yml
f = open("DTDA-config.yml", 'r')
yml_dict = yaml.load(f, Loader=yaml.FullLoader)

loc=None
dum_imax = 36  # lon
dum_jmax = 36  # lat
# ========= dataset for plot =========
cGENIEGrid = yml_dict['core']['data_dir'] + '/data_misc/cGENIEGrid.csv'
cGENIEGrid = pandas.read_csv(cGENIEGrid)
#print(cGENIEGrid)
cGENIEGridB_lat36 = cGENIEGrid['lat']
cGENIEGridB_lon36 = cGENIEGrid['lon']

In [100]:

nexp = yml_dict['core']['nexp']
nens = yml_dict['core']['nens']
datadir_output = yml_dict['core']['datadir_output']
recon_period = yml_dict['core']['recon_period']
recon_timescale = yml_dict['core']['recon_timescale_interval']
recon_period_full = np.arange(recon_period[0],recon_period[1]+1,recon_timescale)
recon_period_len = recon_period_full.shape[0]

# for saving DA product Xa
Xa_output   = np.full((dum_imax, dum_jmax,nens,recon_period_len),np.nan)

# NetCDF file name
nc_filename = datadir_output + '/' + nexp + '.nc'
# read preprior HDF5 file
dir_proxies = yml_dict['proxies'][yml_dict['proxies']['use_from'][0]]['datadir_proxy'] +'/'+ yml_dict['proxies'][yml_dict['proxies']['use_from'][0]]['dbversion']
hdf5name = dir_proxies + '.hdf5'

with h5py.File(hdf5name, 'r') as f:
    Xb = f.get('Xb')
    #Xb = ma.masked_where(Xb>9.9e+36, Xb)
    obvalue_full = f.get('obvalue')
    Ye_full = f.get('Ye')
    ob_err_full = f.get('ob_err')
    Yevar = f.get('Yevar')
    #print(Xb.shape) # (1296, 150)
    #print(Ye_full.shape) # (150, 1)
    print(obvalue_full)
    ob_len = obvalue_full.shape[0]
    print('recon intervals: {}, obser number {}'.format(recon_period_len,ob_len))
    for reconi in range(recon_period_len):
        for obi in range(ob_len):
            print('recon intervals: {}, obser number {}'.format(reconi,obi))
            obvalue  = obvalue_full[obi, reconi]
            ob_err= ob_err_full[obi, reconi]
            Ye = Ye_full[:,obi]
            #Xa = LMR_DA.enkf_update_array(Xb, obm, Ye, obvar)
            
            # Get ensemble size from passed array: Xb has dims [state vect.,ens. members]
            Nens = Xb.shape[1]
            # ensemble mean background and perturbations
            xbm = np.mean(Xb,axis=1)
            Xbp = np.subtract(Xb,xbm[:,None])  # "None" means replicate in this dimension
            #print(xbm.shape)  #  (1296,)  cGENIE
            #print(Xbp.shape)  # (1296, 150)  cGENIE
            # ensemble mean and variance of the background estimate of the proxy
            mye   = np.mean(Ye)
            varye = np.var(Ye,ddof=1)
            print(mye)    # -4.410062826814151
            print(varye)  # 0.5017489599276731
            # lowercase ye has ensemble-mean removed 
            ye = np.subtract(Ye, mye)
            # innovation
            try:
                innov = obvalue - mye
            except:
                print('innovation error. obvalue = ' + str(obvalue) + ' mye = ' + str(mye))
                print('returning Xb unchanged...')
                #return Xb
            # innovation variance (denominator of serial Kalman gain)
            kdenom = (varye + ob_err)  
            #print('kdenom {}'.format(kdenom)) # 0.5622579034828427
            # numerator of serial Kalman gain (cov(x,Hx))
            kcov = np.dot(Xbp,np.transpose(ye)) / (Nens-1)
            # Option to localize the gain
            if loc is not None:
                kcov = np.multiply(kcov,loc) 
            #print(kcov[36:72])
            #print(kcov.shape)  # (1296, 1)
            # Kalman gain
            kmat = np.divide(kcov, kdenom)
            
            #print(kmat.shape)  # (1296, 1)
            # update ensemble mean
            xam = xbm + np.multiply(kmat,innov)
            #print(xbm.shape)
            #print(xam.shape)
            print('obvalue {}, mye {}, innov {}'.format(obvalue, mye, innov))
            print(kmat[36:52])
            print(xbm[36:52])
            print(xam[36:52])
            
            # update the ensemble members using the square-root approach
            beta = 1./(1. + np.sqrt(ob_err/(varye+ob_err)))
            kmat = np.multiply(beta,kmat)
            ye   = np.array(ye)[np.newaxis]
            kmat = np.array(kmat)[np.newaxis]
            Xap  = Xbp - np.dot(kmat.T, ye)

            # full state
            Xa = np.add(xam[:,None], Xap)
            print(Xa.shape)
            # if masked array, making sure that fill_value = nan in the new array 
            if np.ma.isMaskedArray(Xa): 
                np.ma.set_fill_value(Xa, np.nan)
            
        Xa_output[:,:,:,reconi] = Xa.reshape(dum_imax,dum_jmax,nens)

    print('start writing netCDF')
    nf = Dataset(nc_filename, 'w', format='NETCDF4')

    #Add global attributes
    nf.description = 'DTDA' + nc_filename

    #create a group
    tempgrp = nf.createGroup('DA-sst')
    #Specifying dimensions
    tempgrp.createDimension('lon', len(cGENIEGridB_lat36))
    tempgrp.createDimension('lat', len(cGENIEGridB_lon36))
    z = np.arange(0,1,1)
    tempgrp.createDimension('z', len(z))  # level
    tempgrp.createDimension('nens', nens)  # number of ens
    tempgrp.createDimension('time', recon_period_len)

    # Building variables
    longitude = tempgrp.createVariable('Longitude', 'f4', 'lon')
    # Passing data into variables
    longitude[:] = cGENIEGridB_lon36.values
    
    latitude = tempgrp.createVariable('Latitude', 'f4', 'lat')
    latitude[:] = cGENIEGridB_lat36.values
    
    levels = tempgrp.createVariable('Levels', 'i4', 'z')
    levels[:] = z
    
    XbNC_mean = tempgrp.createVariable('Xb_mean', 'f4', ('lat', 'lon','z'))
    XbNC_mean[:,:,:] = xbm.reshape(dum_jmax,dum_imax,1)
    
    XbNC_variance = tempgrp.createVariable('Xb_variance', 'f4', ('lat', 'lon','z'))
    XbNC_variance[:,:,:] = np.nanvar(Xbp,axis=1).reshape(dum_jmax,dum_imax,1)
    
    XaNC_mean = tempgrp.createVariable('Xa_mean', 'f4', ('lat', 'lon','z','time'))
    XaNC_mean[:,:,:,:] = np.nanmean(Xa_output,axis=2).reshape(dum_jmax,dum_imax,1,recon_period_len)
    
    XaNC_variance = tempgrp.createVariable('Xa_variance', 'f4', ('lat', 'lon','z','time'))
    XaNC_variance[:,:,:,:] = np.nanvar(Xa_output,axis=2).reshape(dum_jmax,dum_imax,1,recon_period_len)
    
    #XbNC_full = tempgrp.createVariable('Xb_full', 'f4', ('lat', 'lon', 'nens', 'z'))
    #XbNC_full[:,:,:,:] = Xb.reshape(dum_jmax,dum_imax,nens,1)
    
    XaNC_full = tempgrp.createVariable('Xa_full', 'f4', ('lat', 'lon', 'nens', 'z','time'))
    XaNC_full[:,:,:,:,:] = Xa_output.reshape(dum_jmax,dum_imax,nens,1,recon_period_len)
    #redvarvalue = tempgrp.createVariable('reducedvariance', 'f4', 'time')
    #nestbestid = tempgrp.createVariable('Time', 'i4', 'time')

    #redvarvalue[:] = XavbsumMax[:]
    #nestbestid = site_id    
    
    #Add local attributes to variable instances
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    levels.units = 'layer'
    XbNC_mean.units = 'degC'
    XbNC_variance.units = 'degC^2'
    XbNC_full.units = 'degC'
    XaNC_full.units = 'degC'
    #variance.warning = 'test ...'
    # Closing the dataset
    nf.close()  # close the new file
    print('end writing netCDF')
print('All Done')

<HDF5 dataset "obvalue": shape (1, 3), type "<f8">
recon intervals: 3, obser number 1
recon intervals: 0, obser number 0
-4.409823591726435
0.5014141483029091
obvalue -3.1044, mye -4.409823591726435, innov 1.3054235917264352
[-2.9208903  -2.92892289 -2.9460591  -2.94903453 -2.94379304 -2.93678963
 -2.92748772 -2.91506768 -2.90378259 -2.89410571  0.         -3.02137897
 -3.01578927 -3.0202552  -2.98689168 -2.96039366]
[1.56283980e+01 1.52720551e+01 1.49483772e+01 1.50130476e+01
 1.48461373e+01 1.49580074e+01 1.50402823e+01 1.52286835e+01
 1.56710208e+01 1.61733990e+01 9.96920997e+36 1.43326772e+01
 1.47175994e+01 1.52907178e+01 1.54298405e+01 1.52353625e+01]
[1.18153988e+01 1.14485700e+01 1.11025221e+01 1.11633083e+01
 1.10032405e+01 1.11242529e+01 1.12186708e+01 1.14232853e+01
 1.18803545e+01 1.23953652e+01 9.96920997e+36 1.03884979e+01
 1.07807169e+01 1.13480055e+01 1.15306817e+01 1.13707948e+01]
(1296, 150)
recon intervals: 1, obser number 0
-4.409823591726435
0.5014141483029091
obva

In [93]:
#nf.close()  # close the new file

In [101]:
nf = Dataset(nc_filename, 'r', format='NETCDF4')
XaNC_mean = nf['DA-sst'].variables['Xa_mean']
XaNC_variance = nf['DA-sst'].variables['Xa_variance']
for reconi in range(recon_period_len):
    Xam1 = XaNC_mean[:,:,0,reconi]
    #Xam1 = Xam1.reshape(dum_jmax*dum_imax,1)
    XaNC_var1 = XaNC_variance[:,:,0,reconi]
    #XaNC_var1 = XaNC_var1.reshape(dum_jmax*dum_imax,1)
    print(XaNC_mean.shape)
    print(Xam1.shape)
    print(np.nanmean(Xam1))
    print(np.sqrt(np.nanmean(XaNC_var1)))
nf.close()  # close the new file

(36, 36, 1, 3)
(36, 36)
26.228313
1.5360518
(36, 36, 1, 3)
(36, 36)
28.597527
1.5360518
(36, 36, 1, 3)
(36, 36)
26.724512
1.5360518
